In [82]:
import pandas as pd 
import json
import requests
import numpy as np
import datetime
from datetime import timedelta
from splinter import Browser
from bs4 import BeautifulSoup


In [83]:
# Pull metadata for Bog Springs Campgraound
bs_url = "https://api.weather.gov/points/31.7276,-110.8754"
response_bs = requests.get(bs_url)
data_bs = response_bs.json()
data_bs
grid_data_bs = data_bs["properties"]["forecastGridData"]
grid_data_bs


'https://api.weather.gov/gridpoints/TWC/91,26'

In [84]:
# Pull Forecast Grid data for BogSprings Campground
bs_forcast_url = grid_data_bs
response_bs_forecast = requests.get(bs_forcast_url)
data_bs_forecast = response_bs_forecast.json()

data_bs_forecast
lat_bs = data_bs_forecast["geometry"]["coordinates"][0][0][1]
lat_bs
lng_bs = data_bs_forecast["geometry"]["coordinates"][0][0][0]
lng_bs
elevation_bs = data_bs_forecast["properties"]["elevation"]["value"]
elevation_bs



# Create a Dataframe for Bog Springs Campground
bs_df = pd.DataFrame({"id": 1,
              "campground": "Bog Springs",
              "lat": [lat_bs],
              "lon": [lng_bs],
              "elevation": [elevation_bs],
              "nws_meta_url": [bs_url],
              "nws_grid_url": [grid_data_bs],
              "forest_url":"https://www.fs.usda.gov/recarea/coronado/recreation/camping-cabins/recarea/?recid=25732&actid=29",
              "campsite_url": "https://www.fs.usda.gov/Internet/FSE_MEDIA/fseprd746637.jpg",
              "fire_danger": "Very High",
              "map_code": '<iframe src="https://www.google.com/maps/embed?pb=!1m18!1m12!1m3!1d3393.5714340164473!2d-110.87758868361043!3d31.72759998130141!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m3!1m2!1s0x86d6970db0a5e44d%3A0x1b48084e4d6db970!2sBog%20Springs%20Campground!5e0!3m2!1sen!2sus!4v1626560932236!5m2!1sen!2sus" width="600" height="450" style="border:0;" allowfullscreen="" loading="lazy"></iframe>' 
                    })
bs_df


,id,campground,lat,lon,elevation,nws_meta_url,nws_grid_url,forest_url,campsite_url,fire_danger,map_code
0,1,Bog Springs,31.739186,-110.889864,1502.9688,"https://api.weather.gov/points/31.7276,-110.8754","https://api.weather.gov/gridpoints/TWC/91,26",https://www.fs.usda.gov/recarea/coronado/recre...,https://www.fs.usda.gov/Internet/FSE_MEDIA/fse...,Very High,"<iframe src=""https://www.google.com/maps/embed..."


In [193]:
# Pull Grid Data for Bog Springs CampGround
temp = []
for i in data_bs_forecast["properties"]["temperature"]["values"]:
    temp.append(i)
temp_df = pd.DataFrame(temp)
temp_df
# Temperature conversion to Degree Fahrenheit
temp_df['degF'] = (temp_df['value'] * 9 / 5) + 32
temp_df

# validTime Column split to date and time for Temperature
date_temp = temp_df['validTime'].str.split('T', n=1, expand=True)
time_temp = date_temp[1].str.split('+', n=1, expand=True)
time_temp
temp_df['date_temp'] = date_temp[0]
temp_df['time_temp'] = time_temp[0]

temp_df['date_time_temp'] = temp_df['date_temp'] + ' ' + temp_df['time_temp']
temp_df['date_time_temp'] = pd.to_datetime(temp_df['date_time_temp'])

# Pulling only today + 3 days worth of data
time_delta_temp = datetime.datetime.strptime(temp_df['date_temp'][0],"%Y-%m-%d") + timedelta(days = 4)
temp_df['times_temp'] = time_delta_temp.strftime("%Y-%m-%d")
temp_df = temp_df.loc[temp_df['date_temp'] < temp_df['times_temp']]
temp_df
# temp_df.dtypes


# ##### Pulling Wind Speed ########
wind_speed = []
for i in data_bs_forecast["properties"]["windSpeed"]["values"]:
    wind_speed.append(i) 
windSpeed_df = pd.DataFrame(wind_speed) 
windSpeed_df

# Converting KM/hour to Miles/hour
windSpeed_df['miles/hour'] = windSpeed_df['value'] * 0.621371
windSpeed_df

# validTime Column split to date and time for wind Speed
date_ws = windSpeed_df['validTime'].str.split('T', n=1, expand=True)
time_ws = date_ws[1].str.split('+', n=1, expand=True)
time_ws
windSpeed_df['date_ws'] = date_ws[0]
windSpeed_df['time_ws'] = time_ws[0]

windSpeed_df['date_time_ws'] = windSpeed_df['date_ws'] + ' ' + windSpeed_df['time_ws']
windSpeed_df['date_time_ws'] = pd.to_datetime(windSpeed_df['date_time_ws'])

# Pulling only today + 3 days worth of data
time_delta_ws = datetime.datetime.strptime(windSpeed_df['date_ws'][0],"%Y-%m-%d") + timedelta(days = 4)
windSpeed_df['times_ws'] = time_delta_ws.strftime("%Y-%m-%d")
windSpeed_df = windSpeed_df.loc[windSpeed_df['date_ws'] < windSpeed_df['times_ws']]
windSpeed_df
# windSpeed_df.dtypes


# ##### Pulling Wind Gusts ########
wind_gust = []
for i in data_bs_forecast["properties"]["windGust"]["values"]:
    wind_gust.append(i) 
wind_gust_df = pd.DataFrame(wind_gust) 
wind_gust_df

# Converting KM/hour to Miles/hour
wind_gust_df['m/h'] = wind_gust_df['value'] * 0.621371
wind_gust_df

# # validTime Column split to date and time for wind Gusts
date_wg = wind_gust_df['validTime'].str.split('T', n=1, expand=True)
time_wg = date_wg[1].str.split('+', n=1, expand=True)
time_wg
wind_gust_df['date_wg'] = date_wg[0]
wind_gust_df['time_wg'] = time_wg[0]

wind_gust_df['date_time_wg'] = wind_gust_df['date_wg'] + ' ' + wind_gust_df['time_wg']
wind_gust_df['date_time_wg'] = pd.to_datetime(wind_gust_df['date_time_wg'])
wind_gust_df

# Pulling only today + 3 days worth of data
time_delta_wg = datetime.datetime.strptime(wind_gust_df['date_wg'][0],"%Y-%m-%d") + timedelta(days = 4)
wind_gust_df['times_wg'] = time_delta_wg.strftime("%Y-%m-%d")
wind_gust_df = wind_gust_df.loc[wind_gust_df['date_wg'] < wind_gust_df['times_wg']]
wind_gust_df
# wind_gust_df.dtypes

# # ##### Pulling Probability Precipitation ########
prob_precip = []
for i in data_bs_forecast["properties"]["probabilityOfPrecipitation"]["values"]:
    prob_precip.append(i) 
prob_precip_df = pd.DataFrame(prob_precip) 
prob_precip_df

# # validTime Column split to date and time for Probability Precipitation
date_pp = prob_precip_df['validTime'].str.split('T', n=1, expand=True)
time_pp = date_pp[1].str.split('+', n=1, expand=True)
time_pp
prob_precip_df['date_pp'] = date_pp[0]
prob_precip_df['time_pp'] = time_pp[0]

prob_precip_df['date_time_pp'] = prob_precip_df['date_pp'] + ' ' + prob_precip_df['time_pp']
prob_precip_df['date_time_pp'] = pd.to_datetime(prob_precip_df['date_time_pp'])
prob_precip_df

# Pulling only today + 3 days worth of data
time_delta_pp = datetime.datetime.strptime(prob_precip_df['date_pp'][0],"%Y-%m-%d") + timedelta(days = 4)
prob_precip_df['times_pp'] = time_delta_pp.strftime("%Y-%m-%d")
prob_precip_df = prob_precip_df.loc[prob_precip_df['date_pp'] < prob_precip_df['times_pp']]
prob_precip_df
# prob_precip_df.dtypes

# # ##### Pulling quantity Precipitation ########
qty_precip = []
for i in data_bs_forecast["properties"]["quantitativePrecipitation"]["values"]:
    qty_precip.append(i) 
qty_precip_df = pd.DataFrame(qty_precip) 
qty_precip_df

# # validTime Column split to date and time for quantity Precipitation
date_qp = qty_precip_df['validTime'].str.split('T', n=1, expand=True)
time_qp = date_qp[1].str.split('+', n=1, expand=True)
time_qp
qty_precip_df['date_qp'] = date_qp[0]
qty_precip_df['time_qp'] = time_qp[0]

qty_precip_df['date_time_qp'] = qty_precip_df['date_qp'] + ' ' + qty_precip_df['time_qp']
qty_precip_df['date_time_qp'] = pd.to_datetime(qty_precip_df['date_time_qp'])
qty_precip_df

# Pulling only today + 3 days worth of data
time_delta_qp = datetime.datetime.strptime(qty_precip_df['date_qp'][0],"%Y-%m-%d") + timedelta(days = 4)
qty_precip_df['times_qp'] = time_delta_qp.strftime("%Y-%m-%d")
qty_precip_df = qty_precip_df.loc[qty_precip_df['date_qp'] < qty_precip_df['times_qp']]
qty_precip_df
# qty_precip_df.dtypes


bs_grid_df = pd.DataFrame({"id":1,
        "campground": "Bog Springs",
        "forecasted_temperature_degF": temp_df['degF'],
        "forecastTime_temperature": temp_df['date_time_temp'],
        "forecasted_windSpeed_miles_per_h": windSpeed_df['miles/hour'],
        "forecastTime_windSpeed": windSpeed_df['date_time_ws'],
        "forecasted_windGust_miles_per_h": wind_gust_df['m/h'],
        "forecastTime_windGust": wind_gust_df['date_time_wg'],
        "forecasted_probabilityOfPrecipitation": prob_precip_df['value'],
        "forecastTime_probabilityOfPrecipitation": prob_precip_df['date_time_pp'],
        "forecasted_quantityOfPrecipitation_mm": qty_precip_df['value'],
        "forecastTime_quantityOfPrecipitation": qty_precip_df['date_time_qp'],
       })
bs_grid_df
# bs_grid_df.dtypes

,id,campground,forecasted_temperature_degF,forecastTime_temperature,forecasted_windSpeed_miles_per_h,forecastTime_windSpeed,forecasted_windGust_miles_per_h,forecastTime_windGust,forecasted_probabilityOfPrecipitation,forecastTime_probabilityOfPrecipitation,forecasted_quantityOfPrecipitation_mm,forecastTime_quantityOfPrecipitation
0,1,Bog Springs,84.0,2021-07-21 17:00:00,5.753895,2021-07-21 17:00:00,10.357012,2021-07-21 17:00:00,3.0,2021-07-21 17:00:00,0.000,2021-07-21 17:00:00
1,1,Bog Springs,87.0,2021-07-21 18:00:00,8.055454,2021-07-21 18:00:00,11.507791,2021-07-21 18:00:00,44.0,2021-07-21 18:00:00,1.016,2021-07-21 18:00:00
2,1,Bog Springs,86.0,2021-07-21 22:00:00,6.904675,2021-07-21 20:00:00,12.658570,2021-07-21 19:00:00,29.0,2021-07-22 00:00:00,0.000,2021-07-22 00:00:00
3,1,Bog Springs,87.0,2021-07-21 23:00:00,8.055454,2021-07-21 21:00:00,11.507791,2021-07-22 00:00:00,5.0,2021-07-22 06:00:00,0.000,2021-07-22 06:00:00
4,1,Bog Springs,84.0,2021-07-22 00:00:00,6.904675,2021-07-21 22:00:00,9.206233,2021-07-22 01:00:00,6.0,2021-07-22 12:00:00,0.000,2021-07-22 12:00:00
5,1,Bog Springs,83.0,2021-07-22 01:00:00,5.753895,2021-07-22 00:00:00,10.357012,2021-07-22 03:00:00,64.0,2021-07-22 18:00:00,14.732,2021-07-22 18:00:00
6,1,Bog Springs,80.0,2021-07-22 02:00:00,6.904675,2021-07-22 06:00:00,9.206233,2021-07-22 04:00:00,56.0,2021-07-23 00:00:00,2.540,2021-07-23 00:00:00
7,1,Bog Springs,78.0,2021-07-22 03:00:00,8.055454,2021-07-22 08:00:00,10.357012,2021-07-22 08:00:00,29.0,2021-07-23 06:00:00,0.762,2021-07-23 06:00:00
8,1,Bog Springs,76.0,2021-07-22 04:00:00,6.904675,2021-07-22 09:00:00,9.206233,2021-07-22 09:00:00,50.0,2021-07-23 12:00:00,9.398,2021-07-23 12:00:00
9,1,Bog Springs,75.0,2021-07-22 05:00:00,5.753895,2021-07-22 12:00:00,8.055454,2021-07-22 11:00:00,85.0,2021-07-23 18:00:00,15.748,2021-07-23 18:00:00


In [6]:
# Pull metadata for Rose Canyon Campgraound
rc_url = "https://api.weather.gov/points/32.395,-110.6911"
response_rc = requests.get(rc_url)
data_rc = response_rc.json()
data_rc
grid_data_rc = data_rc["properties"]["forecastGridData"]
grid_data_rc

'https://api.weather.gov/gridpoints/TWC/101,54'

In [8]:
# Pull Forecast Grid data for Rose Canyon Campground
rc_forcast_url = grid_data_rc
response_rc_forecast = requests.get(rc_forcast_url)
data_rc_forecast = response_rc_forecast.json()
data_rc_forecast
lat_rc = data_rc_forecast["geometry"]["coordinates"][0][0][1]
lat_rc
lng_rc = data_rc_forecast["geometry"]["coordinates"][0][0][0]
lng_rc
elevation_rc = data_rc_forecast["properties"]["elevation"]["value"]
elevation_rc

# Create a Dataframe for Rose Canyon Campground
rc_df = pd.DataFrame({"id": 2,
              "campground": "Rose Canyon",
              "lat": [lat_rc],
              "lon": [lng_rc],
              "elevation": [elevation_rc],
              "nws_meta_url": [rc_url],
              "nws_grid_url": [grid_data_rc],
              "forest_url":"https://www.fs.usda.gov/recarea/coronado/recreation/camping-cabins/recarea/?recid=25698&actid=29",
              "campsite_url": "https://cdn.recreation.gov/public/2019/06/20/00/19/232284_beeddff5-c966-49e2-93a8-c63c1cf21294_700.jpg",
            #   "nws_meta_json":[data_rc],
            #   "nws_grid_json": [data_rc_forecast],
              "fire_danger": "Very High",
              "map_code": '<iframe src="https://www.google.com/maps/embed?pb=!1m18!1m12!1m3!1d3368.97130566869!2d-110.70672358360277!3d32.39313088108983!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m3!1m2!1s0x86d6400421614087%3A0xb6cfb84a4b05c95b!2sRose%20Canyon%20Campground!5e0!3m2!1sen!2sus!4v1626560965073!5m2!1sen!2sus" width="600" height="450" style="border:0;" allowfullscreen="" loading="lazy"></iframe>' 
                    })
rc_df



,id,campground,lat,lon,elevation,nws_meta_url,nws_grid_url,forest_url,campsite_url,fire_danger,map_code
0,2,Rose Canyon,32.395779,-110.71132,2132.076,"https://api.weather.gov/points/32.395,-110.6911","https://api.weather.gov/gridpoints/TWC/101,54",https://www.fs.usda.gov/recarea/coronado/recre...,https://cdn.recreation.gov/public/2019/06/20/0...,Very High,"<iframe src=""https://www.google.com/maps/embed..."


In [9]:
# Pull Grid Data for Rose Canyon CampGround
temp_rc = []
for i in data_rc_forecast["properties"]["temperature"]["values"]:
    temp_rc.append(i)
temp_rc_df = pd.DataFrame(temp_rc)
temp_rc_df

wind_speed_rc = []
for i in data_rc_forecast["properties"]["windSpeed"]["values"]:
    wind_speed_rc.append(i) 
windSpeed_rc_df = pd.DataFrame(wind_speed_rc) 
windSpeed_rc_df

wind_gust_rc = []
for i in data_rc_forecast["properties"]["windGust"]["values"]:
    wind_gust_rc.append(i) 
wind_gust_rc_df = pd.DataFrame(wind_gust_rc) 
wind_gust_rc_df

prob_precip_rc = []
for i in data_rc_forecast["properties"]["probabilityOfPrecipitation"]["values"]:
    prob_precip_rc.append(i) 
prob_precip_rc_df = pd.DataFrame(prob_precip_rc) 
prob_precip_rc_df

qty_precip_rc = []
for i in data_rc_forecast["properties"]["quantitativePrecipitation"]["values"]:
    qty_precip_rc.append(i) 
qty_precip_rc_df = pd.DataFrame(qty_precip_rc) 
qty_precip_rc_df

rc_grid_df = pd.DataFrame({"id":2,
        "campground": "Rose Canyon",
        "forecasted_temperature_degC": temp_rc_df['value'],
        "forecastTime_temperature": temp_rc_df['validTime'],
        "forecasted_windSpeed_km_per_h": windSpeed_rc_df['value'],
        "forecastTime_windSpeed": windSpeed_rc_df['validTime'],
        "forecasted_windGust_km_per_h": wind_gust_rc_df['value'],
        "forecastTime_windGust": wind_gust_rc_df['validTime'],
        "forecasted_probabilityOfPrecipitation": prob_precip_rc_df['value'],
        "forecastTime_probabilityOfPrecipitation": prob_precip_rc_df['validTime'],
        "forecasted_quantityOfPrecipitation_mm": qty_precip_rc_df['value'],
        "forecastTime_quantityOfPrecipitation": qty_precip_rc_df['validTime'],
       })
rc_grid_df

,id,campground,forecasted_temperature_degC,forecastTime_temperature,forecasted_windSpeed_km_per_h,forecastTime_windSpeed,forecasted_windGust_km_per_h,forecastTime_windGust,forecasted_probabilityOfPrecipitation,forecastTime_probabilityOfPrecipitation,forecasted_quantityOfPrecipitation_mm,forecastTime_quantityOfPrecipitation
0,2,Rose Canyon,23.888889,2021-07-21T17:00:00+00:00/PT1H,9.260,2021-07-21T17:00:00+00:00/PT2H,16.668,2021-07-21T17:00:00+00:00/PT1H,6.0,2021-07-21T17:00:00+00:00/PT1H,0.000,2021-07-21T17:00:00+00:00/PT1H
1,2,Rose Canyon,26.111111,2021-07-21T18:00:00+00:00/PT2H,11.112,2021-07-21T19:00:00+00:00/PT1H,18.520,2021-07-21T18:00:00+00:00/PT1H,51.0,2021-07-21T18:00:00+00:00/PT6H,1.016,2021-07-21T18:00:00+00:00/PT6H
2,2,Rose Canyon,25.555556,2021-07-21T20:00:00+00:00/PT1H,9.260,2021-07-21T20:00:00+00:00/PT7H,20.372,2021-07-21T19:00:00+00:00/PT1H,31.0,2021-07-22T00:00:00+00:00/PT6H,0.254,2021-07-22T00:00:00+00:00/PT6H
3,2,Rose Canyon,26.111111,2021-07-21T21:00:00+00:00/PT1H,11.112,2021-07-22T03:00:00+00:00/PT1H,22.224,2021-07-21T20:00:00+00:00/PT1H,9.0,2021-07-22T06:00:00+00:00/PT6H,0.000,2021-07-22T06:00:00+00:00/PT6H
4,2,Rose Canyon,25.555556,2021-07-21T22:00:00+00:00/PT1H,9.260,2021-07-22T04:00:00+00:00/PT10H,20.372,2021-07-21T21:00:00+00:00/PT3H,7.0,2021-07-22T12:00:00+00:00/PT6H,0.000,2021-07-22T12:00:00+00:00/PT6H
...,...,...,...,...,...,...,...,...,...,...,...,...
108,2,Rose Canyon,21.111111,2021-07-29T01:00:00+00:00/PT1H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,2,Rose Canyon,20.000000,2021-07-29T02:00:00+00:00/PT1H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,2,Rose Canyon,18.888889,2021-07-29T03:00:00+00:00/PT1H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,2,Rose Canyon,18.333333,2021-07-29T04:00:00+00:00/PT8H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Pull metadata for Spencer Canyon Campgraound
sc_url = "https://api.weather.gov/points/32.4186,-110.7383"
response_sc = requests.get(sc_url)
data_sc = response_sc.json()
data_sc
grid_data_sc = data_sc["properties"]["forecastGridData"]
grid_data_sc

'https://api.weather.gov/gridpoints/TWC/100,56'

In [12]:
# Pull Forecast Grid data for Spencer Canyon Campground
sc_forcast_url = grid_data_sc
response_sc_forecast = requests.get(sc_forcast_url)
data_sc_forecast = response_sc_forecast.json()
data_sc_forecast
lat_sc = data_sc_forecast["geometry"]["coordinates"][0][0][1]
lat_sc
lng_sc = data_sc_forecast["geometry"]["coordinates"][0][0][0]
lng_sc
elevation_sc = data_sc_forecast["properties"]["elevation"]["value"]
elevation_sc

# Create a Dataframe for Spencer Canyon Campground
sc_df = pd.DataFrame({"id": 3,
              "campground": "Spencer Canyon",
              "lat": [lat_sc],
              "lon": [lng_sc],
              "elevation": [elevation_sc],
              "nws_meta_url": [sc_url],
              "nws_grid_url": [grid_data_sc],
              "forest_url":"https://www.fs.usda.gov/recarea/coronado/recreation/camping-cabins/recarea/?recid=25710&actid=29",
              "campsite_url": "https://www.fs.usda.gov/Internet/FSE_MEDIA/fseprd746608.jpg",
            #   "nws_meta_json":[data_sc],
            #   "nws_grid_json": [data_sc_forecast],
              "fire_danger": "Very High",
              "map_code": '<iframe src="https://www.google.com/maps/embed?pb=!1m18!1m12!1m3!1d3368.0814680369876!2d-110.74302428360251!3d32.41697578108229!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m3!1m2!1s0x86d61515ca1f56fd%3A0x242e26b2f2f72242!2sSpencer%20Canyon%20Campground!5e0!3m2!1sen!2sus!4v1626560995515!5m2!1sen!2sus" width="600" height="450" style="border:0;" allowfullscreen="" loading="lazy"></iframe>' 
                    })
sc_df


,id,campground,lat,lon,elevation,nws_meta_url,nws_grid_url,forest_url,campsite_url,fire_danger,map_code
0,3,Spencer Canyon,32.438137,-110.744165,2193.036,"https://api.weather.gov/points/32.4186,-110.7383","https://api.weather.gov/gridpoints/TWC/100,56",https://www.fs.usda.gov/recarea/coronado/recre...,https://www.fs.usda.gov/Internet/FSE_MEDIA/fse...,Very High,"<iframe src=""https://www.google.com/maps/embed..."


In [13]:
# Pull Grid Data for Spencer Canyon CampGround
temp_sc = []
for i in data_sc_forecast["properties"]["temperature"]["values"]:
    temp_sc.append(i)
temp_sc_df = pd.DataFrame(temp_sc)
temp_sc_df

wind_speed_sc = []
for i in data_sc_forecast["properties"]["windSpeed"]["values"]:
    wind_speed_sc.append(i) 
windSpeed_sc_df = pd.DataFrame(wind_speed_sc) 
windSpeed_sc_df

wind_gust_sc = []
for i in data_sc_forecast["properties"]["windGust"]["values"]:
    wind_gust_sc.append(i) 
wind_gust_sc_df = pd.DataFrame(wind_gust_sc) 
wind_gust_sc_df

prob_precip_sc = []
for i in data_sc_forecast["properties"]["probabilityOfPrecipitation"]["values"]:
    prob_precip_sc.append(i) 
prob_precip_sc_df = pd.DataFrame(prob_precip_sc) 
prob_precip_sc_df

qty_precip_sc = []
for i in data_sc_forecast["properties"]["quantitativePrecipitation"]["values"]:
    qty_precip_sc.append(i) 
qty_precip_sc_df = pd.DataFrame(qty_precip_sc) 
qty_precip_sc_df

sc_grid_df = pd.DataFrame({"id":3,
        "campground": "Spencer Canyon",
        "forecasted_temperature_degC": temp_sc_df['value'],
        "forecastTime_temperature": temp_sc_df['validTime'],
        "forecasted_windSpeed_km_per_h": windSpeed_sc_df['value'],
        "forecastTime_windSpeed": windSpeed_sc_df['validTime'],
        "forecasted_windGust_km_per_h": wind_gust_sc_df['value'],
        "forecastTime_windGust": wind_gust_sc_df['validTime'],
        "forecasted_probabilityOfPrecipitation": prob_precip_sc_df['value'],
        "forecastTime_probabilityOfPrecipitation": prob_precip_sc_df['validTime'],
        "forecasted_quantityOfPrecipitation_mm": qty_precip_sc_df['value'],
        "forecastTime_quantityOfPrecipitation": qty_precip_sc_df['validTime'],
       })
sc_grid_df

,id,campground,forecasted_temperature_degC,forecastTime_temperature,forecasted_windSpeed_km_per_h,forecastTime_windSpeed,forecasted_windGust_km_per_h,forecastTime_windGust,forecasted_probabilityOfPrecipitation,forecastTime_probabilityOfPrecipitation,forecasted_quantityOfPrecipitation_mm,forecastTime_quantityOfPrecipitation
0,3,Spencer Canyon,23.888889,2021-07-21T17:00:00+00:00/PT1H,5.556,2021-07-21T17:00:00+00:00/PT1H,16.668,2021-07-21T17:00:00+00:00/PT1H,5.0,2021-07-21T17:00:00+00:00/PT1H,0.000,2021-07-21T17:00:00+00:00/PT1H
1,3,Spencer Canyon,25.555556,2021-07-21T18:00:00+00:00/PT2H,7.408,2021-07-21T18:00:00+00:00/PT1H,18.520,2021-07-21T18:00:00+00:00/PT2H,56.0,2021-07-21T18:00:00+00:00/PT6H,1.270,2021-07-21T18:00:00+00:00/PT6H
2,3,Spencer Canyon,25.000000,2021-07-21T20:00:00+00:00/PT1H,9.260,2021-07-21T19:00:00+00:00/PT2H,20.372,2021-07-21T20:00:00+00:00/PT2H,34.0,2021-07-22T00:00:00+00:00/PT6H,0.254,2021-07-22T00:00:00+00:00/PT6H
3,3,Spencer Canyon,25.555556,2021-07-21T21:00:00+00:00/PT1H,7.408,2021-07-21T21:00:00+00:00/PT4H,18.520,2021-07-21T22:00:00+00:00/PT6H,10.0,2021-07-22T06:00:00+00:00/PT6H,0.000,2021-07-22T06:00:00+00:00/PT6H
4,3,Spencer Canyon,25.000000,2021-07-21T22:00:00+00:00/PT1H,9.260,2021-07-22T01:00:00+00:00/PT3H,16.668,2021-07-22T04:00:00+00:00/PT1H,7.0,2021-07-22T12:00:00+00:00/PT6H,0.000,2021-07-22T12:00:00+00:00/PT6H
...,...,...,...,...,...,...,...,...,...,...,...,...
109,3,Spencer Canyon,22.222222,2021-07-29T00:00:00+00:00/PT1H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,3,Spencer Canyon,20.555556,2021-07-29T01:00:00+00:00/PT1H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,3,Spencer Canyon,18.888889,2021-07-29T02:00:00+00:00/PT1H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,3,Spencer Canyon,18.333333,2021-07-29T03:00:00+00:00/PT1H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df = pd.concat([bs_df, rc_df, sc_df])

df.to_csv('finaldf.csv')
df
# df1 = df.set_index('id')
# df1

,id,campground,lat,lon,elevation,nws_meta_url,nws_grid_url,forest_url,campsite_url,fire_danger,map_code
0,1,Bog Springs,31.739186,-110.889864,1502.9688,"https://api.weather.gov/points/31.7276,-110.8754","https://api.weather.gov/gridpoints/TWC/91,26",https://www.fs.usda.gov/recarea/coronado/recre...,https://www.fs.usda.gov/Internet/FSE_MEDIA/fse...,Very High,"<iframe src=""https://www.google.com/maps/embed..."
0,2,Rose Canyon,32.395779,-110.711320,2132.0760,"https://api.weather.gov/points/32.395,-110.6911","https://api.weather.gov/gridpoints/TWC/101,54",https://www.fs.usda.gov/recarea/coronado/recre...,https://cdn.recreation.gov/public/2019/06/20/0...,Very High,"<iframe src=""https://www.google.com/maps/embed..."
0,3,Spencer Canyon,32.438137,-110.744165,2193.0360,"https://api.weather.gov/points/32.4186,-110.7383","https://api.weather.gov/gridpoints/TWC/100,56",https://www.fs.usda.gov/recarea/coronado/recre...,https://www.fs.usda.gov/Internet/FSE_MEDIA/fse...,Very High,"<iframe src=""https://www.google.com/maps/embed..."


In [15]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from postgres_pswd import host, database, username, passwd

# Define how many sectors to display in the output of the sector analysis
output='top3'

# Create engine to mutual_funds database
engine_startup = "postgresql://" + username + ":" + passwd + "@" + host + "/" + database + "?sslmode=require"

engine = create_engine(engine_startup)

# reflect the existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# View all of the classes that automap found
Base.classes.keys()

# Create our session (link) from Python to the DB
session = Session(bind=engine)


In [16]:
# Create table "camp_wx"

engine.execute('DROP TABLE IF EXISTS camp_wx CASCADE; \
		CREATE TABLE "camp_wx" ( \
		"id" int   NOT NULL, \
		"campground" varchar(255)   NOT NULL, \
		"lat" float   NOT NULL, \
		"lon" float   NOT NULL, \
		"elevation" float   NOT NULL, \
		"nws_meta_url" varchar(2000)   NOT NULL, \
		"nws_grid_url" varchar(2000)   NOT NULL, \
		"forest_url" varchar(2000)   NOT NULL, \
		"campsite_url" varchar(2000)   NOT NULL, \
		"nws_meta_json" varchar(65535)   NOT NULL, \
		"nws_grid_json" varchar(65535)   NOT NULL, \
		"fire_danger" varchar(50)   NOT NULL, \
		"map_code" varchar(20000)   NOT NULL \
	);')

In [17]:
# upload df to camp_wx

df.to_sql('camp_wx', engine, if_exists='replace')


In [18]:
# create table cg_bog_spring

engine.execute('DROP TABLE IF EXISTS cg_bog_spring CASCADE; \
		CREATE TABLE "cg_bog_spring" ( \
		"id" int   NOT NULL, \
		"campground" varchar(255)   NOT NULL, \
		"forecasted_temperature_degC" float, \
		"forecastTime_temperature" varchar(255), \
		"forecasted_windSpeed_km_per_h" float, \
		"forecastTime_windSpeed" varchar(255), \
		"forecasted_windGust_km_per_h" float, \
		"forecastTime_windGust" varchar(255), \
		"forecasted_probabilityOfPrecipitation" float, \
		"forecastTime_probabilityOfPrecipitation" varchar(255), \
		"forecasted_quantityOfPrecipitation_mm" float, \
		"forecastTime_quantityOfPrecipitation" varchar(255) \
	);')

In [19]:
# upload bs_grid_df to cg_bog_spring

bs_grid_df.to_sql('cg_bog_spring', engine, if_exists='replace')

In [20]:
# create table cg_rose_canyon

engine.execute('DROP TABLE IF EXISTS cg_rose_canyon CASCADE; \
		CREATE TABLE "cg_rose_canyon" ( \
		"id" int   NOT NULL, \
		"campground" varchar(255)   NOT NULL, \
		"forecasted_temperature_degC" float, \
		"forecastTime_temperature" varchar(255), \
		"forecasted_windSpeed_km_per_h" float, \
		"forecastTime_windSpeed" varchar(255), \
		"forecasted_windGust_km_per_h" float, \
		"forecastTime_windGust" varchar(255), \
		"forecasted_probabilityOfPrecipitation" float, \
		"forecastTime_probabilityOfPrecipitation" varchar(255), \
		"forecasted_quantityOfPrecipitation_mm" float, \
		"forecastTime_quantityOfPrecipitation" varchar(255) \
	);')

In [21]:
# upload rc_grid_df to cg_rose_canyon

rc_grid_df.to_sql('cg_rose_canyon', engine, if_exists='replace')

In [22]:
# create table cg_spencer_canyon

engine.execute('DROP TABLE IF EXISTS cg_spencer_canyon CASCADE; \
		CREATE TABLE "cg_spencer_canyon" ( \
		"id" int   NOT NULL, \
		"campground" varchar(255)   NOT NULL, \
		"forecasted_temperature_degC" float, \
		"forecastTime_temperature" varchar(255), \
		"forecasted_windSpeed_km_per_h" float, \
		"forecastTime_windSpeed" varchar(255), \
		"forecasted_windGust_km_per_h" float, \
		"forecastTime_windGust" varchar(255), \
		"forecasted_probabilityOfPrecipitation" float, \
		"forecastTime_probabilityOfPrecipitation" varchar(255), \
		"forecasted_quantityOfPrecipitation_mm" float, \
		"forecastTime_quantityOfPrecipitation" varchar(255) \
	);')

In [23]:
# upload sc_grid_df to cg_spencer_canyon

sc_grid_df.to_sql('cg_spencer_canyon', engine, if_exists='replace')